# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('../../../../Datasets/previsao_de_renda.csv', sep = ',')

In [60]:
df.info()

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

#### 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [61]:
from sklearn.model_selection import train_test_split

# utilizando patsy para criar a matriz de design e dummies
import patsy

# criando a matriz de design

y, X = patsy.dmatrices('renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data = df, return_type = 'dataframe') 

# separando treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)


#### 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [62]:
#utilizando elastic net para fazer o ridge imprimindo o summary do statsmodels no final

import statsmodels.api as sm
import numpy as np
from statsmodels.tools.tools import pinv_extended

alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# diciionário para armazenar os resultados de cada alpha 
resultados_ridge = {}

for a in alpha:
    model = sm.OLS(y_train, X_train)
    res = model.fit_regularized(method = 'elastic_net', alpha = a, L1_wt = 0)
    pinv_wexog,_ = pinv_extended(model.wexog)
    normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
    summary = sm.regression.linear_model.OLSResults(model,res.params,normalized_cov_params)
    print(summary.summary())
    
    # armazenando o R2 e R2 ajustado
    resultados_ridge[a] = {'R2': summary.rsquared, 'R2 ajustado': summary.rsquared_adj}
for a in resultados_ridge:
    print(f'Alpha: {a} - R2: {resultados_ridge[a]["R2"]} - R2 ajustado: {resultados_ridge[a]["R2 ajustado"]}')  
    

#### 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [63]:
# diciionário para armazenar os resultados de cada alpha 
resultados_lasso = {}

for a in alpha:
    model = sm.OLS(y_train, X_train)
    res = model.fit_regularized(method = 'elastic_net', alpha = a, L1_wt = 1)
    pinv_wexog,_ = pinv_extended(model.wexog)
    normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
    summary = sm.regression.linear_model.OLSResults(model,res.params,normalized_cov_params)
    # armazenando o R2 e R2 ajustado
    resultados_lasso[a] = {'R2': summary.rsquared, 'R2 ajustado': summary.rsquared_adj}

for a in resultados_lasso:
    print(f'Alpha: {a} - R2: {resultados_lasso[a]["R2"]} - R2 ajustado: {resultados_lasso[a]["R2 ajustado"]}')


#### 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

In [64]:
# utilizando o stepwise para fazer a seleção de variáveis


def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out=0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    """
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.float64)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_train, y_train['renda'])
print('resulting features:')
print(variaveis)

In [65]:
# rodando o modelo com as variáveis selecionadas
model = sm.OLS(y_train, X_train[variaveis])
res = model.fit()
print(res.summary())

y_pred = res.predict(X_test[variaveis])

from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

#### 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?



In [66]:
# comparando os resultados
melhor_ridge = max(resultados_ridge, key = lambda x: resultados_ridge[x]['R2 ajustado'])
melhor_lasso = max(resultados_lasso, key = lambda x: resultados_lasso[x]['R2 ajustado'])


print(f'Melhor modelo Ridge: {melhor_ridge} - R2: {resultados_ridge[melhor_ridge]["R2"]} - R2 ajustado: {resultados_ridge[melhor_ridge]["R2 ajustado"]}')
print(f'Melhor modelo Lasso: {melhor_lasso} - R2: {resultados_lasso[melhor_lasso]["R2"]} - R2 ajustado: {resultados_lasso[melhor_lasso]["R2 ajustado"]}')
print(f'Melhor modelo Stepwise:  - R2: {res.rsquared} - R2 ajustado: {res.rsquared_adj}')

 O modelo stepwise foi o que obteve o melhor resultado, com um R2 ajustado, mas o Ridge obteve o melhor R2, então a escolha do melhor modelo depende do que se deseja otimizar. A diferença é marginal demais para se escolher um modelo em detrimento do outro sem uma análise mais aprofundada observando o problema que se deseja resolver.

#### 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veze se consegue inserir alguma transformação ou combinação de variáveis.

Na célula abaixo, criei versões logarítmicas das features numéricas e adicionei essas features na fórmula patsy. Em seguida, rodei o modelo stepwise novamente para selecionar as variáveis mais relevantes entre normais e logaritmas, com o target renda também em log. O modelo resultante obteve um R2 ajustado  de 0.243 melhorando 0.02 do modelo anterior.

In [67]:
# Criar versões logarítmicas das features numéricas
for col in df.select_dtypes(include=[np.number]).columns:
    df['log_' + col] = np.log1p(df[col])

df['log_renda'] = np.log1p(df['renda'])

# Combinar as features originais com as versões logarítmicas na fórmula patsy
formula = 'log_renda ~ ' + ' + '.join(df.columns.drop(['data_ref', 'index', 'renda', 'log_renda']))

print(formula)
# Criar matrizes de design usando patsy
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# stepwise
variaveis = stepwise_selection(X_train, y_train['log_renda'])

# rodando o modelo com as variáveis selecionadas    
model = sm.OLS(y_train, X_train[variaveis])
res = model.fit()
print(res.summary())

#### 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [86]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# busca de hiperparâmetros com random search e cross validation
param_dist = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None] + list(np.arange(1, 100)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'min_weight_fraction_leaf': uniform(0, 0.5),
    'max_leaf_nodes': [None] + list(np.arange(2, 100)),
}

model = DecisionTreeRegressor(random_state=42)

search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=500, cv=5, scoring='r2', random_state=42, n_jobs=-1)

search.fit(X_train[variaveis], y_train)

# R2 na base de teste 
y_pred = search.predict(X_test[variaveis])
r2_score(y_test, y_pred)

O Melhor modelo segue sendo o stepwise, com um R2 ajustado de 0.243, mesmo realizando uma busca de hiperparâmetros com uma árvore de regressão, o R2 não melhorou.